##IPerl Notebook to explore classes that import the AtomGroupRole in Support of: 
###HackaMol: an object-oriented Modern Perl library for molecular hacking on multiple scales
Demian Riccardi, Jerry M. Parks, Alex Johs, and Jeremy C. Smith


####Description
This notebook explores how to define and use groups of atoms. The HackaMol::AtomGroupRole provides many attributes and methods that are shared by the HackaMol::AtomGroup, HackaMol::Bond, HackaMol::Angle, and HackaMol::Dihedral.  The HackaMol::Molecule class, which is discussed in another notebook, is a subclass of the HackaMol::AtomGroup class. All documentation can be loaded on the commandline (e.g. perldoc HackaMol::AtomGroupRole) or at [Metacpan](https://metacpan.org/pod/HackaMol). This notebook focuses on the HackaMol::AtomGroup class and is written using the iPerl kernel, written by Zaki Mughal, of the iPython notebook.

In [1]:
use Modern::Perl;
use HackaMol;
use Time::HiRes qw(time);

###HackaMol::AtomGroup
As of version 0.013, the HackaMol::AtomGroup class adds a single method, the calculation of the radius of gyration, to the HackaMol::AtomGroupRole. In the following cell, PDBID: 2CBA is downloaded from the Protein DataBank into the local directory. It is then loaded into an array of atoms, filtering out water molecules, using an molecular object builder. 

In [2]:
use LWP::Simple;
my $pdb   = "2CBA.pdb"; 
my $fpdb  = getstore("http://pdb.org/pdb/files/$pdb","$pdb");

my $builder = HackaMol->new ;

my @atoms  = grep {$_->resname !~ m/HOH/} $builder->read_file_atoms($pdb); # read atoms, filter out the water
my $bigG   = HackaMol::AtomGroup->new(atoms=>[@atoms]); # we will use this below in section on trans-rots

HackaMol::AtomGroup=HASH(0x7f8d6f73f640)


Next, the array of atoms is passed the molecular object builder, which generates an array of HackaMol::AtomGroup objects using the resid entry of the pdb file. 

In [3]:
my @resids = $builder->group_by_atom_attr( 'resid', @atoms);

Let's loop over the first 5 groups and do a little printing:

In [4]:
foreach my $res ( @resids[0 .. 4] ) {
    my $atom = $res->get_atoms(0);
    printf("%-4s %i\n", $atom->resname, $atom->resid);
    printf("Atom Bin: %-7s %8.2f COM: %8.3f %8.3f %8.3f\n", $res->bin_atoms_name,$res->total_mass, @{$res->COM}); 
    $res->print_xyz;
} 

HIS  10
Atom Bin: ON3C6     130.08 COM:   13.544   -9.638   10.990
10

  N  11.404000  -9.765000  11.500000
  C  12.463000  -8.762000  11.679000
  C  12.017000  -7.363000  12.042000
  O  12.802000  -6.530000  12.486000
  C  13.379000  -8.776000  10.385000
  C  14.166000 -10.089000  10.351000
  N  13.974000 -11.102000   9.450000
  C  15.153000 -10.501000  11.194000
  C  14.773000 -12.125000   9.726000
  N  15.510000 -11.780000  10.792000
LEU  100
Atom Bin: ONC6      102.07 COM:   -5.051  -19.663   11.384
8

  N  -4.910000 -18.133000  10.544000
  C  -5.741000 -19.255000  10.907000
  C  -6.937000 -18.778000  11.767000
  O  -6.844000 -17.754000  12.436000
  C  -4.941000 -20.287000  11.725000
  C  -3.632000 -20.803000  11.117000
  C  -2.755000 -21.333000  12.233000
  C  -4.076000 -21.852000  10.129000
ASP  101
Atom Bin: O3NC4     110.05 COM:   -9.960  -19.504   12.071
8

  N  -7.974000 -19.607000  11.817000
  C  -9.164000 -19.145000  12.601000
  C  -8.963000 -19.022000  14.099000
  O  -9.73

The group_by_atom_attr method of the builder can group by any attribute. Flexible definitions of atomic groups facilitates analysis and modeling. AtomGroup objects can also be made on the fly, directly.  As a simple example, let's coarse-grain a molecule above using the $C_\alpha$ atom and the center of mass of the sidechain (only if it's beyond a cutoff distance).  

In [5]:
my $t1 = time;
my $rcut = 3.0;

my $coarse_mol = HackaMol::AtomGroup->new();

foreach my $res ( @resids) {

    my @cas =  grep {$_->name eq 'CA'} $res->all_atoms;
    next unless @cas;
    
    $coarse_mol->push_atoms( @cas );     
    my $sidechain  =  HackaMol::AtomGroup->new(atoms=> [
                                                    grep {$_->name ne 'CA'}
                                                    grep {$_->name ne 'O'}
                                                    grep {$_->name ne 'N'}
                                                    grep {$_->name ne 'C'} 
                                                    grep {$_->Z != 1 } $res->all_atoms
                                                ]
    );
    next unless $sidechain->count_atoms;

    my $scat_com = HackaMol::Atom->new(Z=>54, resname=>$cas[0]->resname, coords=>[$sidechain->COM]);

    $coarse_mol->push_atoms($scat_com)  if($cas[0]->distance($scat_com) >= $rcut);

}

my $dt = time - $t1;

printf ("Time coarse-graining: %3.3fs\n",$dt);

$coarse_mol->print_xyz;

Time coarse-graining: 0.036s
342

  C  12.463000  -8.762000  11.679000
 Xe  14.505596 -10.766214  10.306081
  C  -5.741000 -19.255000  10.907000
  C  -9.164000 -19.145000  12.601000
  C  -7.721000 -19.661000  16.096000
  C  -5.894000 -16.350000  16.159000
 Xe  -2.803924 -17.229182  14.969242
  C  -6.301000 -12.825000  14.863000
  C  -7.831000 -10.707000  17.682000
  C  -5.705000  -8.300000  19.686000
 Xe  -4.637849  -5.604095  17.971699
  C  -7.661000  -8.372000  22.951000
 Xe  -8.383859  -5.394868  23.485027
  C  -8.269000 -11.848000  24.429000
  C -10.943000 -12.940000  26.904000
  C  10.176000  -5.737000  12.225000
  C  -9.697000 -15.364000  29.601000
  C  -6.819000 -15.972000  27.120000
 Xe  -4.174635 -15.970029  29.251333
  C  -9.280000 -16.909000  24.330000
 Xe -12.325151 -17.802561  22.712490
  C  -8.271000 -15.481000  20.933000
 Xe  -4.553386 -15.958261  20.845457
  C -10.933000 -14.765000  18.223000
 Xe -13.764236 -14.131207  20.740061
  C -10.474000 -14.936000  14.445000
  C 

GLOB(0x7f8d6a82d808)


###Translations and rotations of groups
Translation and rotation transformations are provided by the AtomGroupRole.  This simplifies the transformations of groups internal to a molecule.  Let's see how this works with some sidechain rotations about the $C_\alpha$-$C_\beta$ bond. In the following we print out a pdb with a movie of the sidechain rotations, 30 degrees at a time up to 360.

In [6]:
my $fh = $bigG->print_pdb('sidechain_movie.pdb');

foreach my $ang (0 .. 11){
    foreach my $res ( @resids) {

        my @cacb =  grep {$_->name eq 'CA' or $_->name eq 'CB'} $res->all_atoms;
        next unless @cacb == 2;

        my $sidechain  =  HackaMol::AtomGroup->new(atoms=> [
                                                    grep {$_->name ne 'CA'}
                                                    grep {$_->name ne 'O'}
                                                    grep {$_->name ne 'N'}
                                                    grep {$_->name ne 'C'} 
                                                    grep {$_->Z != 1 } $res->all_atoms
                                                ]
        );
        next unless $sidechain->count_atoms > 1;
        my ($axle) = $builder->build_bonds(@cacb);
        $sidechain->rotate($axle->bond_vector,30,$axle->get_atoms(1)->xyz);
    }
    $bigG->print_pdb($fh);
}
    
$fh->close;

1


###Clean up files created in this notebook

In [7]:
`rm sidechain_movie.pdb`;
`rm $pdb`;